In [9]:
import uproot as u
import numpy as np
import pyhf
import matplotlib.pyplot as plt 

In [10]:
# Test option
tests = False

flat_uncertainty=False
uncertainty = 0.5
nominal_eps = 1e-3 
obs_eps = []

# Corrections for time window and flux uncertainty

fraction_outside = 1.1 
flux_uncert = 0.22 

# Array to contain expected values 
obs_eps = []
exp_eps = []
exp_minus_two = []
exp_minus_one = []
exp_plus_one = []
exp_plus_two = []
CLb_values = []

scaling_list = [] 
dm_type = "fermion"
alpha = 0.1
ratio = "0.6"
tag = "CNN"


if(alpha==1.0):
    scaling = 0.05
else:
    scaling = 0.05


base_dir_run1 = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/run1_samples/"
base_dir_run3 = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/run3_samples/"

signal_dir_run1 = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/run1_signal/"
signal_dir_run3 = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/run3_signal/"


signal_run1 = u.open(signal_dir_run1 + dm_type + "_ratio_" + ratio + "_signal_hist_run1_" + tag + ".root")
signal_run3 = u.open(signal_dir_run3 + dm_type + "_ratio_" + ratio + "_signal_hist_run3_" + tag + ".root")

bkg_run1 = u.open(base_dir_run1 + "background_hist_run1_" + tag + ".root")
bkg_run3 = u.open(base_dir_run3 + "background_hist_run3_" + tag + ".root")

In [11]:
if(ratio == "0.6"):
    #masses = ["0.01", "0.02", "0.03", "0.04", "0.05", "0.06", "0.07", "0.08", "0.09", "0.1", "0.2", "0.3", "0.4"]
    masses = ["0.05"]
else:
    masses = ["0.010", "0.020", "0.030", 
              "0.040", "0.050", "0.060", 
              "0.065", "0.070", "0.075", 
              "0.080", "0.085", "0.090", 
              "0.095", "0.100", "0.105", 
              "0.110", "0.115", "0.120", "0.125"]
    
print("Background events run1: {nevts:.2f}".format( nevts= np.sum(bkg_run1["bkg_total_hist"].values())))
print("Background events run3: {nevts:.2f}".format( nevts= np.sum(bkg_run3["bkg_total_hist"].values()))) 
print("\n")
print("Data events run1: {nevts:.2f}".format( nevts= np.sum(bkg_run1["data_hist"].values())))
print("Data events run3: {nevts:.2f}".format( nevts= np.sum(bkg_run3["data_hist"].values()))) 

Background events run1: 222.15
Background events run3: 447.76


Data events run1: 226.00
Data events run3: 383.00


In [13]:
if(not tests):


    delimiter=0
    pot_uncert = 0.02 # 2% POT uncertainty 


    total_bkg = np.sum(bkg_run1["bkg_total_hist"].values()) + np.sum(bkg_run3["bkg_total_hist"].values())

    total_run1 = np.sum(bkg_run1["bkg_total_hist"].values())
    total_run3 = np.sum(bkg_run3["bkg_total_hist"].values())

    n_back = bkg_run1["bkg_total_hist"].values().tolist()[delimiter:]
    n_back.extend(bkg_run3["bkg_total_hist"].values().tolist()[delimiter:])


    n_data = bkg_run1["data_hist"].values().tolist()[delimiter:]
    n_data.extend(bkg_run3["data_hist"].values().tolist()[delimiter:])


    if(flat_uncertainty):
        back_sigma =  (bkg_run1["bkg_total_hist"].values()*uncertainty).tolist()
        back_sigma.extend((bkg_run3["bkg_total_hist"].values()*uncertainty).tolist())
    else:
        back_sigma =  (bkg_run1["bkg_total_uncert"].values()).tolist()[delimiter:]
        back_sigma.extend((bkg_run3["bkg_total_uncert"].values()).tolist()[delimiter:])

    if alpha == 1.0:
        scaling_a1 = (1./pow(0.1,3))
    else:
        scaling_a1 = 1. 

    for mass in masses:

        print("Processing mass: " + mass)
        print("Signal events run1: {nevts:.5f}".format( nevts= np.sum(signal_run1["signal_"+ mass].values()*scaling_a1)))
        print("Signal events run3: {nevts:.5f}".format( nevts= np.sum(signal_run3["signal_"+ mass].values()*scaling_a1)))

        total_sig = np.sum(signal_run1["signal_"+mass].values()*scaling_a1) + np.sum(signal_run3["signal_"+mass].values()*scaling_a1)
        total_sig_run1 = np.sum(signal_run1["signal_"+mass].values()*scaling_a1)
        total_sig_run3 =  np.sum(signal_run3["signal_"+mass].values()*scaling_a1)
        
        factor = scaling*(((total_run1/total_sig_run1)+ (total_run3/total_sig_run3))/2.)
        total_sig_adjusted = np.sum(signal_run1["signal_"+mass].values()*scaling_a1*factor) + np.sum(signal_run3["signal_"+mass].values()*scaling_a1*factor)
        
        scaling_list.append(factor)
        

        print("Scaling factor: ", factor)


        # Factor scalings is introduced to have a signal strength between 0 and 10
        # We correct by the fraction of events outside the beam window
        n_sig_run1 = (signal_run1["signal_"+mass].values()*scaling_a1*fraction_outside*factor).tolist()[delimiter:]
        n_sig_run3 = (signal_run3["signal_"+mass].values()*scaling_a1*fraction_outside*factor).tolist()[delimiter:]
        n_sig_run1.extend(n_sig_run3)


        # Total signal errors are in %, need to convert them to absolute errors for shapesys modifier 
        # We correct by the fraction of events outside the beam window
        sigma_sig_run1 = (signal_run1["signal_total_error_"+mass].values()/100.)*(signal_run1["signal_"+mass].values()*scaling_a1*fraction_outside*factor)
        sigma_sig_run3 = (signal_run3["signal_total_error_"+mass].values()/100.)*(signal_run3["signal_"+mass].values()*scaling_a1*fraction_outside*factor)


        sigma_sig_run1 = (sigma_sig_run1).tolist()[delimiter:]
        sigma_sig_run3 = (sigma_sig_run3).tolist()[delimiter:]
        sigma_sig_run1.extend(sigma_sig_run3)



        model = pyhf.Model(
            {
          "channels": [
            {
              "name": "singlechannel",
              "samples": [
                {
                  "name": "signal",
                  "data": n_sig_run1,
                  "modifiers": [
                    {"name": "mu", "type": "normfactor", "data": None}, #This is the scaling which is to be calculated
                    {"name": "uncorr_siguncrt", "type": "shapesys", "data": sigma_sig_run1},
                    {"name": "pot_correlated", "type": "normsys", "data": {"hi":1. + (pot_uncert), "lo": 1. - (pot_uncert)}},
                    {"name": "flux_correlated", "type": "normsys", "data": {"hi":1. + flux_uncert, "lo": 1. -  flux_uncert}},
                  ]
                },
                {
                  "name": "background",
                  "data": n_back,
                  "modifiers": [
                    {"name": "uncorr_bkguncrt", "type": "shapesys", "data": back_sigma},
                    {"name": "pot_correlated", "type": "normsys", "data": {"hi":1. + pot_uncert, "lo": 1 - pot_uncert}},
                  ]
                }
              ]
            }
          ]
        }
        )

        obs = n_data + model.config.auxdata
        poi_values = np.linspace(0., 10., 100)
        obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upperlimit(
            obs, model, poi_values, level=0.1, return_results=True)
        
        
                
        CLs_value, p_values, CLs_band = pyhf.infer.hypotest(obs_limit, obs, model, 
                                                           return_expected_set=True, 
                                                           return_tail_probs=True)
        
        print(f"Upper limit (obs): μ = {obs_limit:.4f}")
        print(f"Upper limit (exp): μ = {exp_limits[2]:.4f}")
        print(f"p-value: = {p_values[1]:.2f}")

        obs_epsilon = (nominal_eps**2)*np.sqrt(obs_limit*factor)
        exp_epsilon = (nominal_eps**2)*np.sqrt(exp_limits[2]*factor)
        exp_two_down = (nominal_eps**2)*np.sqrt(exp_limits[0]*factor)
        exp_one_down = (nominal_eps**2)*np.sqrt(exp_limits[1]*factor)
        exp_one_up = (nominal_eps**2)*np.sqrt(exp_limits[3]*factor)
        exp_two_up = (nominal_eps**2)*np.sqrt(exp_limits[4]*factor)

        obs_eps.append(obs_epsilon)
        exp_eps.append(exp_epsilon)
        exp_minus_two.append(exp_two_down)
        exp_minus_one.append(exp_one_down)
        exp_plus_one.append(exp_one_up)
        exp_plus_two.append(exp_two_up)
        CLb_values.append(p_values[1])
        print(f"Upper limit (obs): epsilon2 = {obs_epsilon}")
        print(f"Upper limit (exp): epsilon2 = {exp_epsilon}")
        print(f"Upper limit +2sigma: epsilon2 = {exp_two_up}")
        print(f"Upper limit +1sigma: epsilon2 = {exp_one_up}")
        print(f"Upper limit -1sigma: epsilon2 = {exp_one_down}")
        print(f"Upper limit -2sigma: epsilon2 = {exp_two_down}")
        print("\n")

Processing mass: 0.05
Signal events run1: 719.79909
Signal events run3: 1811.24287
Scaling factor:  0.013895948257267832
Upper limit (obs): μ = 0.9823
Upper limit (exp): μ = 1.0235
p-value: = 0.46
Upper limit (obs): epsilon2 = 1.1683072656879228e-07
Upper limit (exp): epsilon2 = 1.19260136072126e-07
Upper limit +2sigma: epsilon2 = 1.8887470273039212e-07
Upper limit +1sigma: epsilon2 = 1.4990247887448502e-07
Upper limit -1sigma: epsilon2 = 9.747581000639355e-08
Upper limit -2sigma: epsilon2 = 8.249504397010253e-08




In [ ]:
print(CLs_band)

In [ ]:
# CLs vs mu
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from pyhf.contrib.viz import brazil
figure_size=(20,13)

fig = plt.figure(figsize=figure_size,dpi=300)
ax = fig.add_subplot(111)

for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(2.5)  # change width
    ax.spines[axis].set_color('black')    # change color
    
    
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.set_title("Hypothesis Test", size=40)
ax.set_xlim(0.1,2.0)
#plt.grid(alpha=0.5)
plt.tick_params(axis="y", which='major', direction="in",length=20,width=2.0,pad=10)
plt.tick_params(axis="y", which='minor', direction="in",length=10,width=2.0,labelleft=False)
plt.tick_params(axis="x", which='major', direction="in",length=20,width=2.0,pad=10)
plt.tick_params(axis="x", which='minor', direction="in",length=10,width=2.0,labelbottom=False)
plt.xticks(size=25)
plt.yticks(size=25)
artists = brazil.plot_results(poi_values, results,test_size=0.1, ax=ax)
ax.set_ylabel(r'$\mathrm{CL}_{\mathrm{s}}$',size=40)
ax.set_xlabel(r'$\mu$',size=40)
ax.legend(fontsize=35)
plots_dir = "/home/lmlepin/Desktop/Plots_DT_Drive/2023/sensitivity_plots/"
plt.savefig(plots_dir + "CLs_scan_values.png",bbox_inches="tight")
plt.savefig(plots_dir + "CLs_scan_values.pdf",bbox_inches="tight")
plt.show()

In [ ]:
if(not tests):


    import pandas as pd
    from matplotlib import patheffects

    limits_dir = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/"
    plots_dir = "/home/lmlepin/Desktop/Plots_DT_Drive/2023/sensitivity_plots/"
    sensitivity_files = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/sensitivity_files/"

    df = pd.read_csv(limits_dir + "dark_tridents_current_limits.csv")
    df_b = pd.read_csv(limits_dir + "dark_tridents_current_limits_1.csv")
    df_babar = pd.read_csv(limits_dir + "babar_paper.csv")
    df_na = pd.read_csv(limits_dir + "NA48_2.csv")


    if(ratio=="0.6"):
        dp_masses = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4]
    else:
        dp_masses = [0.010, 0.020, 0.030, 
              0.040, 0.050, 0.060, 
              0.065, 0.070, 0.075, 
              0.080, 0.085, 0.090, 
              0.095, 0.100, 0.105, 
              0.110, 0.115, 0.120, 0.125]

    plt.figure(figsize=(17,14),dpi=300)
    plt.axis([ 1e-2, 1, 1e-11, 1e-5])
    plt.plot(dp_masses,exp_eps,label=r'MicroBooNE',color='red')
    plt.plot(dp_masses,obs_eps,label=r'MicroBooNE',color='black')



    if(dm_type == 'fermion' and alpha == 0.1):
        plt.plot(df['X_LSND'],df['Y_LSND'],'-', label=r'LSND',color='green')
        plt.fill_between(df['X_LSND'],df['Y_LSND'], 1e-5,color='green',alpha=0.2)
        plt.plot(df['X_PLANCK'],df['Y_PLANCK'],'-',color='orange', path_effects=[patheffects.withTickedStroke(spacing=10, angle=280)])
        plt.plot(df['X_PLANCK'],df['Y_PLANCK'],'-', label=r'Planck (fermionic DM)',color='orange')
    elif(dm_type == 'fermion' and alpha == 1.):
        plt.plot(df_b['X_LSND'],df_b['Y_LSND'],'-', label=r'LSND',color='green')
        plt.fill_between(df_b['X_LSND'],df_b['Y_LSND'], 1e-5,color='green',alpha=0.2)
        plt.plot(df_b['X_PLANCK'],df_b['Y_PLANCK'],'-',color='orange', path_effects=[patheffects.withTickedStroke(spacing=10, angle=280)])
        plt.plot(df_b['X_PLANCK'],df_b['Y_PLANCK'],'-', label=r'Planck (fermionic DM)',color='orange')
    else:
        pass


    limit_dict = {'mass':dp_masses,'observed':obs_eps, 'epsilon_squared':exp_eps, 'two_sig_down':exp_minus_two,
                 'one_sig_down':exp_minus_one, 'one_sig_up':exp_plus_one, 'two_sig_up':exp_plus_two,
                 'CLb':CLb_values}
    #df_out = pd.DataFrame.from_dict(limit_dict)
   # df_out.to_csv(sensitivity_files + dm_type + "_" + tag + "_sensitivity_alpha_" + str(alpha) + "_ratio_" + ratio + "_all_runs_full_uncert_signal_flux_0_percent.csv")


    #plt.plot(df['X_BD'],df['Y_BD'],'-', label=r'Beam Dump',color='mediumpurple')
    plt.fill_between(df['X_BD'],df['Y_BD'],1e-11,color='mediumpurple',alpha=0.3)
    #plt.plot(df_babar['X_babar'],np.square(df_babar['Y_babar']),'-', label=r'BaBar',color='blue')
    plt.fill_between(df_babar['X_babar'],np.square(df_babar['Y_babar']), 1e-5,color='blue',alpha=0.2)
    #plt.plot(df_na['X']*1e-3,df_na['Y'],'-', label=r'NA48/2',color='blue')
    plt.fill_between(df_na['X']*1e-3,df_na['Y'], 1e-5,color='blue',alpha=0.3)
    plt.yscale('log')
    plt.xscale('log')
    plt.ylim(1e-11,1e-5)
    plt.xlim(1e-2,5e-1)
    plt.legend(fontsize=20,loc="lower right",shadow=True)
    plt.xticks(size=25)
    plt.yticks(size=25)
    plt.title("Sensitivity for 7.56e20 POT           " + dm_type + " DM: " + r'$\alpha_{D}$ = ' + str(alpha), size = 30, pad=20)
    plt.xlabel(r'$M_{A^{\prime}}$[GeV]',size=30, labelpad=20)
    plt.ylabel(r'$\epsilon^2$',size=30, labelpad=20)
    #plt.minorticks_off()
    #plt.savefig(plots_dir + dm_type + "_" + tag + "_sensitivity_alpha_" + str(alpha) + ".png")

In [ ]:
# Comparison with toy-based calculator 


if(tests):
    masses = ["0.05"]
    delimiter=0
    pot_uncert = 0.02 # 2% POT uncertainty 


    total_bkg = np.sum(bkg_run1["bkg_total_hist"].values()) + np.sum(bkg_run3["bkg_total_hist"].values())

    total_run1 = np.sum(bkg_run1["bkg_total_hist"].values())
    total_run3 = np.sum(bkg_run3["bkg_total_hist"].values())

    n_back = bkg_run1["bkg_total_hist"].values().tolist()[delimiter:]
    n_back.extend(bkg_run3["bkg_total_hist"].values().tolist()[delimiter:])


    n_data = bkg_run1["data_hist"].values().tolist()[delimiter:]
    n_data.extend(bkg_run3["data_hist"].values().tolist()[delimiter:])


    if(flat_uncertainty):
        back_sigma =  (bkg_run1["bkg_total_hist"].values()*uncertainty).tolist()
        back_sigma.extend((bkg_run3["bkg_total_hist"].values()*uncertainty).tolist())
    else:
        back_sigma =  (bkg_run1["bkg_total_uncert"].values()).tolist()[delimiter:]
        back_sigma.extend((bkg_run3["bkg_total_uncert"].values()).tolist()[delimiter:])

    if alpha == 1.0:
        scaling_a1 = (1./pow(0.1,3))
    else:
        scaling_a1 = 1. 

    for mass in masses:

        print("Processing mass: " + mass)
        print("Signal events run1: {nevts:.5f}".format( nevts= np.sum(signal_run1["signal_"+ mass].values()*scaling_a1)))
        print("Signal events run3: {nevts:.5f}".format( nevts= np.sum(signal_run3["signal_"+ mass].values()*scaling_a1)))

        total_sig = np.sum(signal_run1["signal_"+mass].values()*scaling_a1) + np.sum(signal_run3["signal_"+mass].values()*scaling_a1)
        total_sig_run1 = np.sum(signal_run1["signal_"+mass].values()*scaling_a1)
        total_sig_run3 =  np.sum(signal_run3["signal_"+mass].values()*scaling_a1)
        factor = scaling*(((total_run1/total_sig_run1)+ (total_run3/total_sig_run3))/2.)
        total_sig_adjusted = np.sum(signal_run1["signal_"+mass].values()*scaling_a1*factor) + np.sum(signal_run3["signal_"+mass].values()*scaling_a1*factor)
        scaling_list.append(factor)

        print("Scaling factor: ", factor)


        # Factor scalings is introduced to have a signal strength between 0 and 10
        n_sig_run1 = (signal_run1["signal_"+mass].values()*scaling_a1*factor).tolist()[delimiter:]
        n_sig_run3 = (signal_run3["signal_"+mass].values()*scaling_a1*factor).tolist()[delimiter:]
        n_sig_run1.extend(n_sig_run3)


        # Total signal errors are in %, need to convert them to absolute errors for shapesys modifier  
        sigma_sig_run1 = (signal_run1["signal_total_error_"+mass].values()/100.)*(signal_run1["signal_"+mass].values()*scaling_a1*factor)
        sigma_sig_run3 = (signal_run3["signal_total_error_"+mass].values()/100.)*(signal_run3["signal_"+mass].values()*scaling_a1*factor)


        sigma_sig_run1 = (sigma_sig_run1).tolist()[delimiter:]
        sigma_sig_run3 = (sigma_sig_run3).tolist()[delimiter:]
        sigma_sig_run1.extend(sigma_sig_run3)



        model = pyhf.Model(
            {
          "channels": [
            {
              "name": "singlechannel",
              "samples": [
                {
                  "name": "signal",
                  "data": n_sig_run1,
                  "modifiers": [
                    {"name": "mu", "type": "normfactor", "data": None}, #This is the scaling which is to be calculated
                    {"name": "uncorr_siguncrt", "type": "shapesys", "data": sigma_sig_run1},
                    {"name": "pot_correlaated", "type": "normsys", "data": {"hi":1.02, "lo":0.98}},
                  ]
                },
                {
                  "name": "background",
                  "data": n_back,
                  "modifiers": [
                    {"name": "uncorr_bkguncrt", "type": "shapesys", "data": back_sigma},
                    {"name": "pot_correlated", "type": "normsys", "data": {"hi":1.02, "lo":0.98}},
                  ]
                }
              ]
            }
          ]
        }
        )

        obs = n_data + model.config.auxdata

        poi_values = np.linspace(0., 10., 100)
        obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upperlimit(
            obs, model, poi_values, level=0.1, return_results=True)
        
        
        print("Printing upper limits results...")
        print(f"Upper limit (obs): μ = {obs_limit:.4f}")
        print(f"Upper limit (exp): μ = {exp_limits[2]:.4f}")

        obs_epsilon = (nominal_eps**2)*np.sqrt(obs_limit*factor)
        exp_epsilon = (nominal_eps**2)*np.sqrt(exp_limits[2]*factor)
        exp_two_down = (nominal_eps**2)*np.sqrt(exp_limits[0]*factor)
        exp_one_down = (nominal_eps**2)*np.sqrt(exp_limits[1]*factor)
        exp_one_up = (nominal_eps**2)*np.sqrt(exp_limits[3]*factor)
        exp_two_up = (nominal_eps**2)*np.sqrt(exp_limits[4]*factor)

        print(f"Upper limit (obs): epsilon2 = {obs_epsilon}")
        print(f"Upper limit (exp): epsilon2 = {exp_epsilon}")
        print(f"Upper limit +2sigma: epsilon2 = {exp_two_up}")
        print(f"Upper limit +1sigma: epsilon2 = {exp_one_up}")
        print(f"Upper limit -1sigma: epsilon2 = {exp_one_down}")
        print(f"Upper limit -2sigma: epsilon2 = {exp_two_down}")
        print("\n")
        

        
        
        '''
        print("Printing CLs values for the observed value of mu (Asymptotic)...")
        CLs_obs_asymp, CLs_exp_asymp = pyhf.infer.hypotest(exp_limits[2], obs, model, 
                                                           return_expected_set=True)
        
        print(f"Observed CLs = {CLs_obs_asymp}")
        print(f"Expected CLs = {CLs_exp_asymp}")
        print("\n")
        '''
        
        print("Printing CLs values for the observed value of mu (Toys)...")
        CLs_obs_toys, CLs_exp_toys, p_values = pyhf.infer.hypotest(exp_limits[2], obs, model, 
                                                           return_expected_set=True, 
                                                           return_tail_probs=True)
        print(f"Observed CLs = {CLs_obs_toys}")
        print(f"Expected CLs = {CLs_exp_toys}")
        print(f"P-values = {p_values}")
        print("\n")
        
       # print(f"Ratio of both calculators = {CLs_obs_toys/CLs_obs_asymp}")